In [4]:
import pandas as pd
# pd.set_option('display.max_rows', 500)

## Cleaning the datasets from each supermarket for uniformity and consistency

## Woolworths

Data integrity has been checked - audit conducted on 22-Apr-2021. 

When Prices are not empty and Online Only is deselected, the dataset will match all the 'available' products on the website (i.e. ones which have a price and does not say 'unavailable').

In [5]:
df_wow = pd.read_csv('Data/2021-04-23 WOW Data.csv', encoding='utf-8-sig').fillna('')

In [6]:
# Remove commas within Category names - makes exploding easier. 
# I do this "un-compressing" of duplicate items into their corresponding categories here to save space - can easily be done at the scraper stage.

df_wow['Department'] = df_wow['Department'].apply(
    lambda x: x.replace('Meat, Seafood & Deli', 'Meat Seafood & Deli').replace('Dairy, Eggs & Fridge', 'Dairy Eggs & Fridge').replace("'", "").strip("[]")
)

In [7]:
# Change _pk to _ pack in Package Size (i.e. 6pk --> 6 pack)

df_wow['Package Size'] = df_wow['Package Size'].apply(lambda x: x.replace('pk', ' pack')).apply(lambda x: x.replace('l', 'L'))

In [8]:
# Filter:
# - Not Tobacco - because inaccessible behind login wall
# - Price is not empty - which means it's unavilable

filter = (df_wow['Department'] != 'Tobacco Product') & (df_wow['Price'] != '')
df_wow = df_wow.loc[filter]

In [9]:
# Products which are in multiple categories are "exploded" into those categories individually with only that category name attached in the Department column.
'''E.g.: 
Product Name: Tasmanian Heritage Double Brie Cheese Snack 	
Department: ['Meat, Seafood & Deli', 'Dairy, Eggs & Fridge']

The above will be separated out into 2 different lines with the same Product Name but one having 'Meat, Seafood & Deli', 
and the other line with 'Dairy, Eggs & Fridge' in the Department column.
'''

df_wow = df_wow.assign(Department=df_wow.Department.str.split(', ')).explode('Department')

In [10]:
# Liquor is also removed because Woolworths' sales of Liquor are actually BWS stock. Done after exploding to remove the duplicates from Drinks that remain in Liquor.

df_wow = df_wow.loc[df_wow['Department'] != 'Liquor']

In [11]:
# Products with no categories assigned are now named "NOT LISTED"

df_wow['Department'] = df_wow['Department'].apply(lambda x: 'NOT LISTED' if x == '' else x)

In [12]:
# Concatenating Product Name with Package Size to standardize labelling. Essential for next steps, fuzzymatching and inner merge.

df_wow['Brand_Product_Size'] = df_wow['Product Name'] + " " + df_wow['Package Size']

In [13]:
# Re-arrange columns and rename (to reflect what supermarket it belongs to).

df_wow = df_wow[
    ['SKU', 'Brand_Product_Size', 'Brand', 'Product Name', 
     'Price', 'Package Size', 'Price per unit', 'Specials', 
     'Department', 'Online Only', 'New Product', 'Product URL']].reset_index(drop=True)

df_wow = df_wow.rename(columns={
    'Price': 'WOW Price',
    'Package Size': 'WOW Size',
    'Price per unit': 'WOW ppu',
    'Specials': 'WOW Specials',
    'Department': 'WOW Category'
})

In [14]:
df_wow.head()

,SKU,Brand_Product_Size,Brand,Product Name,WOW Price,WOW Size,WOW ppu,WOW Specials,WOW Category,Online Only,New Product,Product URL
0,814139,Em Wholefoods Hemp Oil Cold Pressed 250mL,em wholefoods,Em Wholefoods Hemp Oil Cold Pressed,19.95,250mL,$7.98 / 100ML,,Pantry,,,https://www.woolworths.com.au/shop/productdeta...
1,84972,Happy Little Camper Newborn Natural Nappies 36...,happy little camper,Happy Little Camper Newborn Natural Nappies,14.0,36 pack,$0.39 / 1EA,,Baby,,,https://www.woolworths.com.au/shop/productdeta...
2,95412,Spc Spaghetti Rich Tomato 140g x12 pack,spc,Spc Spaghetti Rich Tomato,10.5,140g x12 pack,$0.63 / 100G,,Pantry,,,https://www.woolworths.com.au/shop/productdeta...
3,99040,Mint Glazed Antipasto Dish Blue each,mint,Mint Glazed Antipasto Dish Blue,2.0,each,,,NOT LISTED,,,https://www.woolworths.com.au/shop/productdeta...
4,808305,Seedlip Grove 42 Alcohol Free 700mL,seedlip,Seedlip Grove 42 Alcohol Free,50.0,700mL,$71.43 / 1L,,Drinks,,,https://www.woolworths.com.au/shop/productdeta...


In [15]:
# Output cleaned data to csv file

df_wow.to_csv('1. Cleaned Data/Woolworths.csv', index=False, encoding='utf-8-sig')

## Coles

Data Integrity is good - audit has been conducted on the scraped data. 
Since, products are constantly uploaded/removed, there is a tiny difference over the 2hr+ time delta between
starting scraping process and completion.

In [16]:
df_coles = pd.read_csv('Data/2021-04-23 Coles Data.csv', encoding='utf-8-sig').fillna('')

In [17]:
# Modify the Unit Size formatting: Replace '1 each' with 'each', and make all the units lower case to be consistent with Woolworths.

df_coles['Unit Size'] = df_coles['Unit Size'].apply(lambda x: x.replace('1 each', 'each').lower()).apply(lambda x: x.replace('l', 'L'))

# Concatenate Brand name, Product Name and Packaging Size into a new column. This column is key for next steps in fuzzymatching and inner merge.

df_coles['Brand_Product_Size'] = df_coles['Brand'] + ' ' + df_coles['Product Name'] + ' ' + df_coles['Unit Size']

In [18]:
# Cleaning up the text output in Category. Replace '--' and '-' with one space. Capitalize first letter(s).

df_coles.Category = df_coles.Category.apply(lambda x: x.replace('--', ' ').replace('-', ' ').title())

In [19]:
# Remove unavailable items and Liquor category - non-alcoholic items should all be duplicated in other categories so there is no information loss.

df_coles = df_coles.loc[(df_coles['Availability'] == 'Available') & (df_coles['Category'] != 'Liquor')]

In [20]:
# Select only the necessary columns and rename with COL tag to reflect Coles.

df_coles = df_coles[['SKU', 'Brand_Product_Size', 'Brand', 'Product Name', 'Current Price', 
                     'Price per unit', 'Unit Size', 'On Special', 'Category', 'URL']]

df_coles = df_coles.rename(columns={
    'Current Price': 'COL Price',
    'Price per unit': 'COL ppu',
    'Unit Size': 'COL Size',
    'On Special': 'COL Specials',
    'Category': 'COL Category',
    'URL': 'Product URL'
})

In [21]:
df_coles.head()

,SKU,Brand_Product_Size,Brand,Product Name,COL Price,COL ppu,COL Size,COL Specials,COL Category,Product URL
0,3980255P,Head & Shoulders Conditioner Smooth & Silky 660mL,Head & Shoulders,Conditioner Smooth & Silky,17.0,$2.58 per 100mL,660mL,,Health Beauty,https://shop.coles.com.au/a/national/product/s...
1,3838775P,Coles Jujube Prepack 400g,Coles,Jujube Prepack,9.9,$24.75 per 1Kg,400g,,Fruit Vegetables,https://shop.coles.com.au/a/national/product/f...
2,3760496P,The Spice Tailor Malabar Biryani 360g,The Spice Tailor,Malabar Biryani,5.5,$1.53 per 100G,360g,,International Foods,https://shop.coles.com.au/a/national/product/t...
3,6046740P,Fantastic Crispy Bacon Noodle Cup 70g,Fantastic,Crispy Bacon Noodle Cup,1.6,$2.29 per 100G,70g,,Pantry,https://shop.coles.com.au/a/national/product/f...
4,403765P,McCain Frozen Ham & Pineapple Family Pizza 500g,McCain,Frozen Ham & Pineapple Family Pizza,5.5,$1.10 per 100G,500g,Specials,Frozen,https://shop.coles.com.au/a/national/product/m...


In [22]:
df_coles.to_csv('1. Cleaned Data/Coles.csv', encoding='utf-8-sig', index=False)

## IGA

In [36]:
df_iga = pd.read_csv('Data/2021-04-23 IGA Data.csv', encoding='utf-8-sig')

In [37]:
# Replace 'gm' with 'g' if found at the end of Product Name.

import re

df_iga['Product Name'] = df_iga['Product Name'].apply(lambda x: re.sub(r'gm$', 'g', x))

# Replace 'pk' using regex rule: Look behind for 'pk' and replace with ' pack'

df_iga['Product Name'] = df_iga['Product Name'].apply(lambda x: re.sub(r'(?<=\d)pk', ' pack', x))

# Capitalise 'l' that follows numbers

df_iga['Product Name'] = df_iga['Product Name'].apply(lambda x: re.sub(r'(?<=\d)l', 'L', x))

In [38]:
# Tidying up formatting. SKU - change float to string, remove decimals. Select only relevant columns, rename to reflect it's IGA.

df_iga.SKU = df_iga['SKU'].astype(str).apply(lambda x: x[:-2])

df_iga = df_iga[['SKU', 'Product Name', 'Price', 'Price per unit', 'Main Category', 'Product URL']]

df_iga = df_iga.rename(columns={
    'Product Name': 'Brand_Product_Size',
    'Price': 'IGA Price',
    'Price per unit': 'IGA ppu',
    'Main Category': 'IGA Category'
})

In [39]:
# Replace '-' with space and capitalize first letter(s) in Category.

df_iga['IGA Category'] = df_iga['IGA Category'].apply(lambda x: x.replace('-', ' ').title())

In [40]:
df_iga.head()

,SKU,Brand_Product_Size,IGA Price,IGA ppu,IGA Category,Product URL
0,9300675009775,Diet Coke Soft Drink 600ml,4.10,2.46 per litre,Drinks,https://igashop.com.au/product/diet-coke-soft-...
1,4155,Granny Smith Apple,0.99,5.50 per kg,Fruit & Veg,https://igashop.com.au/product/granny-smith-ap...
2,4156,Pink Lady Apple,1.20,6.00 per kg,Fruit & Veg,https://igashop.com.au/product/pink-lady-apple/
3,9310023141460,Helga’s Wraps Traditional White 8 Pack 560g,5.55,0.99 per 100g,Bakery,https://igashop.com.au/product/helgas-wraps-tr...
4,9310023141446,Helga’s Mixed Grain Wraps 8 Pack 560g,5.55,0.99 per 100g,Bakery,https://igashop.com.au/product/helgas-mixed-gr...


In [41]:
df_iga.to_csv('1. Cleaned Data/IGA.csv', encoding='utf-8-sig', index=False)

## Aldi

In [ ]:
df_aldi = pd.read_csv('Data/2021-04-23 Aldi Data.csv', encoding='utf-8-sig')

In [ ]:
df_aldi.rename(columns={
    'Sub Category': 'Aldi Category',
    'Product': 'Product Name',
    'Product Page': 'URL'
}, inplace=True)

df_aldi = df_aldi[['Product Name', 'Price', 'Unit Price', 'Aldi Category', 'URL']]

In [ ]:
df_aldi['Aldi Category'] = df_aldi['Aldi Category'].apply(lambda x: 'Dairy Eggs & Fridge' if x == 'fresh product' else x)
df_aldi['Aldi Category'] = df_aldi['Aldi Category'].apply(lambda x: 'Health & Beauty' if x == 'beauty' or x == 'health' else x)
df_aldi['Aldi Category'] = df_aldi['Aldi Category'].apply(lambda x: 'Household' if x == 'laundry household' else x)
df_aldi['Aldi Category'] = df_aldi['Aldi Category'].apply(lambda x: 'Frozen' if x == 'freezer' else x)
df_aldi['Aldi Category'] = df_aldi['Aldi Category'].apply(lambda x: x.title())

In [ ]:
df_aldi['Aldi Category'].value_counts()

In [ ]:
df_aldi.to_csv('1. Cleaned Data/Aldi.csv', encoding='utf-8-sig', index=False)